# Convert to mobile program

In [1]:
# Import all libraries
import torch
import torch.nn as nn
from torchvision import models
import os

from torch.utils.mobile_optimizer import optimize_for_mobile

In [2]:
## Load the models
def load_model(model_name, directory="models"):
    # Initiate pretrained model
    if "resnet18" in model_name:
        model = models.resnet18(weights='IMAGENET1K_V1')
        model.fc = nn.Linear(model.fc.in_features, 2)
    elif "resnet50" in model_name:
        model = models.resnet50(weights='DEFAULT')
        model.fc = nn.Linear(model.fc.in_features, 2)
    elif "alexnet" in model_name:
        model = models.alexnet(weights='DEFAULT')
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)
    elif "efficientnet_b0" in model_name:
        model = models.efficientnet_b0(weights='DEFAULT')
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    elif "efficientnet_b1" in model_name:
        model = models.efficientnet_b1(weights='DEFAULT')
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    elif "efficientnet_b2" in model_name:
        model = models.efficientnet_b2(weights='DEFAULT')
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, 2)
    else:
        raise ValueError("Invalid model name")
    
    # Load the trained model weights
    model.load_state_dict(torch.load(os.path.join(directory, f"{model_name}_model.pt"), weights_only=True))
    return model

In [3]:
# Define model name and load the model
name = "noCuda_mod_efficientnet_b2"
model_name = f"{name}_model_lite"
model = load_model(name)

# Move the model to CPU first
deviceCPU = torch.device("cpu")
model = model.to(deviceCPU)

# Ensure all model parameters are moved to CPU
for param in model.parameters():
    param.data = param.data.to("cpu")
    if param.grad is not None:
        param.grad.data = param.grad.data.to("cpu")

for buffer in model.buffers():
    buffer.data = buffer.data.to("cpu")

# Script the model
scripted_module = torch.jit.script(model)
scripted_model = scripted_module.to(deviceCPU)  # Ensure it is on the CPU

# Optimize for mobile
optimized_scripted_module = optimize_for_mobile(scripted_module)

# Save the optimized lite interpreter model
optimized_scripted_module._save_for_lite_interpreter(f"models/{model_name}.ptl")


In [4]:
try:
    if (not name or not model_name):
        raise Exception("Run the code block above, or define `name` below")
except Exception as _:
    name = "noCuda_efficientnet_b2"
    model_name = f"{name}_model_lite"
    print("Using default name for the model: ", name)
    
# Load the .ptl model and enforce CPU
model = torch.jit.load(f"models/{model_name}.ptl", map_location="cpu")

# Verify the model can run with a dummy input
dummy_input = torch.randn(1, 3, 224, 224)
output = model(dummy_input)
print("Model should successfully output: ", output)

if output is not None:
    print("✅ Model is successfully loaded and running on CPU")

Model should successfully output:  tensor([[-0.1875,  0.2168]])
✅ Model is successfully loaded and running on CPU
